# Shoreline Evolution
Notebook environment to migrate netcdf files to CF compliant zarr

In [ ]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

### Configure OS independent paths

In [ ]:
# Import standard packages
import os
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc
import numpy.ma as ma

# Make root directories importable by appending root to path
cwd = pathlib.Path().resolve()
sys.path.append(os.path.dirname(cwd))

# Get root paths
home = pathlib.Path().home()
root = home.root

# Import custom functionality
from etl import p_drive
from etl.CF_compliancy_checker import check_compliancy, save_compliancy

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11205479-coclico", "data")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    home.joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

In [ ]:
# Project paths & files (manual input)
ds_dir = coclico_data_dir.joinpath("04_shoreline_jrc")
ds_path = ds_dir.joinpath("globalCoastalMorphodynamicsDb.nc")
ds_out_file = "globalCoastalMorphodynamicsDb"
CF_dir = coclico_data_dir.joinpath(r"CF")  # directory to save output CF check files

### Check CF compliancy original NetCDF files

In [ ]:
# open datasets
ds = xr.open_dataset(ds_path)

# check original dataset
ds

In [ ]:
%%capture cap --no-stderr
# check original CF compliancy

check_compliancy(testfile=ds_path, working_dir=CF_dir)

In [ ]:
# save original CF compliancy
save_compliancy(cap, testfile=ds_path, working_dir=CF_dir)

### Make CF compliant alterations to the NetCDF files (dataset dependent)

In [ ]:
# NetCDF attribute alterations

# rename global attribute names
ds.attrs["Project_Acronym"] = ds.attrs.pop("Project Acronym")

# add global attributes
ds.attrs["Conventions"] = "CF-1.8"

# add or change certain variable / coordinate attributes
ds.lat.attrs["units"] = "degrees_north"
ds.lat.attrs["long_name"] = "latitude"
ds.latland.attrs["units"] = "degrees_north"
ds.latsea.attrs["units"] = "degrees_north"
ds.firstlandlat.attrs["units"] = "degrees_north"
ds.lon.attrs["units"] = "degrees_east"
ds.lon.attrs["long_name"] = "longitude"
ds.lonland.attrs["units"] = "degrees_east"
ds.lonsea.attrs["units"] = "degrees_east"
ds.firstlandlon.attrs["units"] = "degrees_east"
ds.spuriousratio.attrs["units"] = "1"
ds.ptid.attrs["units"] = "1"
ds.landid.attrs["units"] = "1"
ds.landfound.attrs["units"] = "1"
ds.segmentid.attrs["units"] = "1"
ds.errorid.attrs["units"] = "1"
# ds.transect.attrs["units"] = "1"
ds.qualityflag.attrs["units"] = "1"
ds.firstYear.attrs["units"] = "yr"
ds.lastYear.attrs["units"] = "yr"
ds.transect.attrs["long_name"] = "number of stations"
ds.activezonetoland.attrs["long_name"] = "active zone to land"
ds.activezonetosea.attrs["long_name"] = "active zone to sea"
ds.landtoactivezone.attrs["long_name"] = "land to active zone"
ds.landtosea.attrs["long_name"] = "land to sea"
ds.ptid.attrs["long_name"] = "stations"
ds.landid.attrs["long_name"] = "country"
ds.errorid.attrs["long_name"] = "error id"

In [ ]:
# NetCDF variable and dimension alterations

# rename or swap dimension names, the latter in case the name already exists as coordinate
ds = ds.rename_dims({"transect": "nstations"})
ds = ds.swap_dims({"ntransect": "nstations"})

# rename variables, if necessary
ds = ds.rename_vars(
    {
        "transect": "nstations",
        "latland": "lat_land",
        "latsea": "lat_sea",
        "lonland": "lon_land",
        "lonsea": "lon_sea",
        "activezonetoland": "active_zone_to_land",
        "activezonetosea": "active_zone_to_sea",
        "landtoactivezone": "land_to_active_zone",
        "landtosea": "land_to_sea",
        "seatoactivezone": "sea_to_active_zone",
        "seatoland": "sea_to_land",
        "spuriousratio": "spurious_ratio",
        "ptid": "stations",
        "landid": "country",
        "firstlandlat": "first_land_lat",
        "firstlandlon": "first_land_lon",
        "landfound": "land_found",
        "segmentid": "coastal_segment",
        "errorid": "error_id",
        "qualityflag": "quality_flag",
        "firstYear": "first_year",
        "lastYear": "last_year",
    }
)

# set some data variables to coordinates to avoid duplication of dimensions in later stage
ds = ds.set_coords(
    [
        "lat",
        "lat_land",
        "lat_sea",
        "lon",
        "lon_land",
        "lon_sea",
        "stations",
        "first_land_lon",
        "first_land_lat",
        "error_id",
        "coastal_segment",
        "quality_flag",
        "spurious_ratio",
        "country",
        "land_found",
        "first_year",
        "last_year",
    ]
)
ds = ds.drop("nstations")

# add epsg
ds.attrs["crs"] = 4326

In [ ]:
# check the xarray dataset, best practice is to have as many as possible bold dimensions (dimension == coordinate name).
# in this way, the Front-End can access the variable directly without having to index the variable first

ds

In [ ]:
# save new .nc files
ds.to_netcdf(path=ds_dir.joinpath(ds_out_file + "_CF.nc"))

### Check CF compliancy altered NetCDF files

In [ ]:
%%capture cap --no-stderr
# check altered CF compliancy

check_compliancy(testfile=ds_dir.joinpath(ds_out_file + "_CF.nc"), working_dir=CF_dir)

In [ ]:
# save altered CF compliancy
save_compliancy(
    cap, testfile=ds_dir.joinpath(ds_out_file + "_CF.nc"), working_dir=CF_dir,
)

### write data to Zarr files

In [ ]:
# export to zarr in write mode (to overwrite if exists)
ds.to_zarr(ds_dir.joinpath("%s.zarr" % ds_out_file), mode="w")